In [ ]:
import os
os.chdir("..")

In [ ]:
import world_bank_data as wb
import pandas as pd
import polars as pl
import missingno as msno
import numpy as np
import pandas as pd
from src.data.data_pull import DataClean
import seaborn as sns
import matplotlib.pyplot as plt

dc = DataClean()

# TODO
- colapse by 5 average
- keep the balance data
- 

In [ ]:
df = dc.pull_wb()
df = df.filter(pl.col("year") >= 1990)
data = pl.read_csv("data/external/countries.csv")
df = df.join(data, on="country", how="inner", validate="m:1")
df = df.select(
    pl.col(
        'year',
        'country',
        'fertility_rate',
        'gdp_per_capita',
        'government_expenditure',
        'trade',
        'life_expectancy',
        'gdp_per_capita_growth',
        'inflation',
        'population_growth',
        'gross_capital_formation',
        'domestic_bank',
        'advanced_economy')
)
df = df.with_columns(
    fertility_rate_dummy=pl.when(pl.col("fertility_rate").is_null()).then(1).otherwise(0),
    gdp_per_capita_dummy=pl.when(pl.col("gdp_per_capita").is_null()).then(1).otherwise(0),
    government_expenditure_dummy=pl.when(pl.col("government_expenditure").is_null()).then(1).otherwise(0),
    trade_dummy=pl.when(pl.col("trade").is_null()).then(1).otherwise(0),
    life_expectancy_dummy=pl.when(pl.col("life_expectancy").is_null()).then(1).otherwise(0),
    gdp_per_capita_growth_dummy=pl.when(pl.col("gdp_per_capita_growth").is_null()).then(1).otherwise(0),
    inflation_dummy=pl.when(pl.col("inflation").is_null()).then(1).otherwise(0),
    population_growth_dummy=pl.when(pl.col("population_growth").is_null()).then(1).otherwise(0),
    gross_capital_formation_dummy=pl.when(pl.col("gross_capital_formation").is_null()).then(1).otherwise(0),
    domestic_bank_dummy=pl.when(pl.col("domestic_bank").is_null()).then(1).otherwise(0),
    advanced_economy_dummy=pl.when(pl.col("advanced_economy").is_null()).then(1).otherwise(0)
)
df2 = df


In [ ]:
msno.matrix(df.to_pandas().sort_values(["year","country"]))

In [ ]:
import matplotlib.pyplot as plt

# Count the number of 1's (missing values) in each dummy column
dummy_columns = [
    "fertility_rate_dummy",
    "gdp_per_capita_dummy",
    "government_expenditure_dummy",
    "trade_dummy",
    "life_expectancy_dummy",
    "gdp_per_capita_growth_dummy",
    "inflation_dummy",
    "population_growth_dummy",
    "gross_capital_formation_dummy",
    "domestic_bank_dummy",
]

# Total number of rows in the dataframe
total_rows = df.height

# Calculate the percentage of missing values (1's) in each dummy column
percent_missing = {}
for column in dummy_columns:
    missing_count = df.filter(pl.col(column) == 1).height  # Count the number of 1's
    percent_missing[column] = (missing_count / total_rows) * 100  # Calculate the percentage

# Sort the percentages for better visualization
sorted_columns = sorted(percent_missing, key=percent_missing.get, reverse=True)
sorted_percentages = [percent_missing[col] for col in sorted_columns]

# Plotting the bar chart
plt.figure(figsize=(10, 6))
plt.barh(sorted_columns, sorted_percentages)
plt.xlabel("Percentage of Missing Values (%)")
plt.title("Percentage of Missing Values for Dummy Columns")
plt.gca().invert_yaxis()  # To have the highest percentage at the top
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# List of dummy columns to check for missing values
dummy_columns = [
    "fertility_rate_dummy",
    "gdp_per_capita_dummy",
    "government_expenditure_dummy",
    "trade_dummy",
    "life_expectancy_dummy",
    "gdp_per_capita_growth_dummy",
    "inflation_dummy",
    "population_growth_dummy",
    "gross_capital_formation_dummy",
    "domestic_bank_dummy",
]

# Group by 'country' and calculate the percentage of missing values for each country
missing_percentage_by_country = []

# Loop through countries
for country in df["country"].unique():
    country_df = df.filter(pl.col("country") == country)
    
    # Calculate percentage of missing values for each dummy column in the country
    country_percentage = {}
    for column in dummy_columns:
        missing_count = country_df.filter(pl.col(column) == 1).height
        total_count = country_df.height
        missing_percentage = (missing_count / total_count) * 100 / 12
        country_percentage[column] = missing_percentage
    
    # Add the country data to the list
    missing_percentage_by_country.append((country, country_percentage))

# Convert the results into a DataFrame for easier plotting
country_names = [entry[0] for entry in missing_percentage_by_country]
country_percentages = [entry[1] for entry in missing_percentage_by_country]

# Create a DataFrame
country_df = pd.DataFrame(country_percentages, index=country_names)

# Split the countries into 4 roughly equal groups
country_chunks = [country_df.iloc[i:i + len(country_df) // 4] for i in range(0, len(country_df), len(country_df) // 4)]

# Create 4 subplots for the 4 chunks
fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharey=True)
axes = axes.flatten()

# Plot each chunk in a different subplot
for idx, chunk in enumerate(country_chunks):
    if idx == 4:
        continue
    chunk.plot(kind='bar', stacked=True, ax=axes[idx], colormap='coolwarm', width=0.8)
    axes[idx].set_title(f"Missing Data for Countries {chunk.index[0]} to {chunk.index[-1]}")
    axes[idx].set_xlabel("Country")
    axes[idx].set_ylabel("Percentage of Missing Values (%)")
    axes[idx].tick_params(axis='x', rotation=45)

# Adjust layout
plt.tight_layout()
plt.legend(title="Variables", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()


In [ ]:
country_list = pl.from_pandas(country_df.reset_index())
min_countries = country_list.with_columns(
    total_pct = pl.concat_list(dummy_columns).list.sum()
).filter(pl.col("total_pct") <= 5).select(pl.col("index")).to_series().to_list()
len(min_countries)

In [ ]:
df = dc.pull_wb()
# df = df.filter(pl.col("year") >= 1990)
data = pl.read_csv("data/external/countries.csv")
df = df.join(data, on="country", how="inner", validate="m:1")
df = df.select(
    pl.col(
        'year',
        'country',
        'fertility_rate',
        'gdp_per_capita',
        'government_expenditure',
        'trade',
        'life_expectancy',
        'gdp_per_capita_growth',
        'inflation',
        'population_growth',
        'gross_capital_formation',
        'domestic_bank',
        'advanced_economy')
)

In [ ]:
extra_contries = ['Iraq', 'Malta', 'Slovak Republic', 'Slovenia','Guinea', 'Mozambique','Namibia', 'Sierra Leone',"Solomon Islands"]
extra2 = ['Albania', 'Armenia', 'Austria', 'Azerbaijan', 'Belarus',
       'Belgium', 'Brunei Darussalam', 'Bulgaria', 'Cambodia', 'Cyprus',
       'Finland', 'France', 'Georgia', 'Germany', 'Greece', 'Ireland',
       'Italy', 'Kazakhstan', 'Kuwait', 'Kyrgyz Republic', 'Luxembourg',
       'Moldova', 'Mongolia', 'North Macedonia', 'Portugal', 'Spain',
       'Ukraine','Benin', 'Nicaragua', 'Oman', 'Romania', 'Uganda',
       'Aruba', 'Haiti', 'Hungary', 'Poland', 'Qatar']

In [ ]:
df = df2.filter(pl.col("country").is_in(min_countries))
df = df.filter(~pl.col("country").is_in(extra_contries))
df = df.filter(~pl.col("country").is_in(extra2))

In [ ]:
data = df.to_pandas()
msno.matrix(data.sort_values(["country", "year"]))

In [ ]:
data = df.to_pandas()
data = data.sort_values(["year", "country"])
columns = [
    "fertility_rate",
    "gdp_per_capita",
    "government_expenditure",
    "trade",
    "life_expectancy",
    "gdp_per_capita_growth",
    "inflation",
    "population_growth",
    "gross_capital_formation",
    "domestic_bank",
    
]
for col in columns:
    print(col)
    data[col] = data.groupby("country")[col].transform(
        lambda group: group.interpolate(method="linear")
    )
data = data.sort_values(["country", "year"])
data = data[data["year"] >= 1990]
msno.matrix(data.sort_values(["country", "year"]))

In [ ]:
data[data["government_expenditure"].isnull()]["country"].unique()

In [ ]:
data.to_csv("data_1990.csv")

In [ ]:
data["advanced_economy"][data["year"] == 2020].sum()

In [ ]:
data["advanced_economy"][data["year"] == 2020].count()

In [ ]:
data[data["domestic_bank"].isnull()]["country"].unique()

In [ ]:
data.sort_values(["country", "year"])

In [ ]:

msno.matrix(df.to_pandas())

In [ ]:
import matplotlib.pyplot as plt

df = df2.filter(pl.col("country").is_in(min_countries))

# Count the number of 1's (missing values) in each dummy column
dummy_columns = [
    "fertility_rate_dummy",
    "gdp_per_capita_dummy",
    "government_expenditure_dummy",
    "trade_dummy",
    "life_expectancy_dummy",
    "gdp_per_capita_growth_dummy",
    "inflation_dummy",
    "population_growth_dummy",
    "gross_capital_formation_dummy",
    "domestic_bank_dummy",
]

# Total number of rows in the dataframe
total_rows = df.height

# Calculate the percentage of missing values (1's) in each dummy column
percent_missing = {}
for column in dummy_columns:
    missing_count = df.filter(pl.col(column) == 1).height  # Count the number of 1's
    percent_missing[column] = (missing_count / total_rows) * 100  # Calculate the percentage

# Sort the percentages for better visualization
sorted_columns = sorted(percent_missing, key=percent_missing.get, reverse=True)
sorted_percentages = [percent_missing[col] for col in sorted_columns]

# Plotting the bar chart
plt.figure(figsize=(10, 6))
plt.barh(sorted_columns, sorted_percentages)
plt.xlabel("Percentage of Missing Values (%)")
plt.title("Percentage of Missing Values for Dummy Columns")
plt.gca().invert_yaxis()  # To have the highest percentage at the top
plt.show()

In [ ]:
temp = df2.filter(pl.col("country").is_in(min_countries))
temp.filter(pl.col("year") == 2015).select(pl.col("advanced_economy")).sum()

In [ ]:
temp.select(pl.col("year")).max()

In [ ]:
temp

In [ ]:
len(min_countries)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd


df = df2.filter(pl.col("country").is_in(min_countries))

# List of dummy columns to check for missing values
dummy_columns = [
    "fertility_rate_dummy",
    "gdp_per_capita_dummy",
    "government_expenditure_dummy",
    "trade_dummy",
    "life_expectancy_dummy",
    "gdp_per_capita_growth_dummy",
    "inflation_dummy",
    "population_growth_dummy",
    "gross_capital_formation_dummy",
    "domestic_bank_dummy",
]

# Group by 'country' and calculate the percentage of missing values for each country
missing_percentage_by_country = []

# Loop through countries
for country in df["country"].unique():
    country_df = df.filter(pl.col("country") == country)
    
    # Calculate percentage of missing values for each dummy column in the country
    country_percentage = {}
    for column in dummy_columns:
        missing_count = country_df.filter(pl.col(column) == 1).height
        total_count = country_df.height
        missing_percentage = (missing_count / total_count) * 100 / 12
        country_percentage[column] = missing_percentage
    
    # Add the country data to the list
    missing_percentage_by_country.append((country, country_percentage))

# Convert the results into a DataFrame for easier plotting
country_names = [entry[0] for entry in missing_percentage_by_country]
country_percentages = [entry[1] for entry in missing_percentage_by_country]

# Create a DataFrame
country_df = pd.DataFrame(country_percentages, index=country_names)

# Split the countries into 4 roughly equal groups
country_chunks = [country_df.iloc[i:i + len(country_df) // 4] for i in range(0, len(country_df), len(country_df) // 4)]

# Create 4 subplots for the 4 chunks
fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharey=True)
axes = axes.flatten()

# Plot each chunk in a different subplot
for idx, chunk in enumerate(country_chunks):
    if idx == 4:
        continue
    chunk.plot(kind='bar', stacked=True, ax=axes[idx], colormap='coolwarm', width=0.8)
    axes[idx].set_title(f"Missing Data for Countries {chunk.index[0]} to {chunk.index[-1]}")
    axes[idx].set_xlabel("Country")
    axes[idx].set_ylabel("Percentage of Missing Values (%)")
    axes[idx].tick_params(axis='x', rotation=45)

# Adjust layout
plt.tight_layout()
plt.legend(title="Variables", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()


In [ ]:
df

In [ ]:
df = df.filter(pl.col("year") >= 1970)
# df.write_csv("data.csv")

In [ ]:
data

In [ ]:
# df = df.filter(pl.col("year") >= 2002)
# for cols in  df.columns:
#     df = df.filter(pl.col(f"{cols}").is_not_null())

# units = df.select("country").unique()

# min_year = df["year"].min()
# max_year = df["year"].max()
# years = pl.DataFrame({"year": list(range(min_year, max_year + 1))})

# full_index = units.join(years, how="cross")

# full_df = full_index.join(df, on=["country", "year"], how="left")

# msno.matrix(full_df.to_pandas().sort_values(["country","year"]))

In [ ]:
msno.matrix(df.to_pandas())

In [ ]:
df = df.filter(pl.col("year") >= 1970)
pivoted = df.pivot(
    values="school_primary",
    index="year",
    on="country"
)
msno.bar(pivoted.to_pandas(), sort='descending')

In [ ]:
pivoted = df.pivot(
    values="school_primary",
    index="country",
    on="year"
)
# pivoted.set_index(['year'], inplace=True)
for year2 in range(2005,2022):
    res = pivoted.filter(pl.col(f"{year2}").is_not_null()).select(pl.col("country")).to_series().unique().to_list()
    for year in range(year2 +1,2022):
        tmp = pivoted.filter(pl.col(f"{year}").is_not_null()).select(pl.col("country")).to_series().unique().to_list()
        res = list(set(res) & set(tmp))
    print(f"Amount of countries {len(res)}")
    print(f"start year {year2}")   

In [ ]:
pivoted

In [ ]:
countries = pivoted.filter(pl.col("2005").is_not_null()).select(pl.col("country")).to_series().unique().to_list()
len(countries)

In [ ]:
# countries2 = pivoted.filter(pl.col("2005").is_not_null()).select(pl.col("country")).to_series().unique().to_list()
# countries2 = pivoted.filter(pl.col("2004").is_not_null()).select(pl.col("country")).to_series().unique().to_list()
# res = list(set(countries2) & set(countries2))

for year2 in range(2005,2022):
    res = pivoted.filter(pl.col(f"{year2}").is_not_null()).select(pl.col("country")).to_series().unique().to_list()
    for year in range(year2 +1,2022):
        tmp = pivoted.filter(pl.col(f"{year}").is_not_null()).select(pl.col("country")).to_series().unique().to_list()
        res = list(set(res) & set(tmp))
    print(f"Amount of countries {len(res)}")
    print(f"start year {year2}")   

In [ ]:
df1 = df
res = pivoted.filter(pl.col(1970).is_not_null()).select(pl.col("country")).to_series().unique().to_list()
for year in range(1970,2024):
    tmp = pivoted.filter(pl.col(f"{year}").is_not_null()).select(pl.col("country")).to_series().unique().to_list()
    res = list(set(res) & set(tmp))
# tmp = pivoted.filter(pl.col(f"").is_not_null()).select(pl.col("country")).to_series().unique().to_list()
df1 = df1.filter(pl.col("country").is_in(res))
# df1 = df1.filter((pl.col("year") >= 2002))
df1 = df1.to_pandas().sort_values(["year","country"])
msno.matrix(df1)

In [ ]:
len(df1.select(pl.col("country")).to_series().unique().to_list())

In [ ]:
df.write_csv("data/processed/dataset2.csv")